In [1]:
!pip install tensorflow opencv-python numpy scikit-learn matplotlib seaborn 



In [4]:
import tensorflow as tf

# Define the path to the saved model
model_path = "C:/RHINO/inference_graph/saved_model"

# Load the model
model = tf.saved_model.load(model_path)
print("Model loaded successfully!")


Model loaded successfully!


In [5]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

# Replace these with actual labels and predictions from your test dataset
y_true = np.array([1, 0, 1, 1, 0, 0, 1, 0, 1, 1])  # Ground truth labels
y_pred = np.array([1, 0, 1, 0, 0, 1, 1, 0, 1, 1])  # Model predictions

# Compute Performance Metrics
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

# Display results
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")


ModuleNotFoundError: No module named 'sklearn'

In [5]:
import tensorflow as tf
import cv2
import numpy as np
import os
from sklearn.metrics import precision_score, recall_score, f1_score

# Load the TensorFlow saved model
model_path = "C:/vehicle-crash-detector/inference_graph/saved_model"  # Adjust path
model = tf.saved_model.load(model_path)
infer = model.signatures["serving_default"]  # Extract inference function

# Define the video path
video_path = "C:\vehicle-crash-detector\test_videos\test (1).mp4"  # Replace with actual video file
cap = cv2.VideoCapture(video_path)

# Define lists to store ground truth and predicted values
y_true = []  # Actual crash labels (1 = crash, 0 = no_crash)
y_pred = []  # Model predictions

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # Stop when the video ends

    frame_count += 1

    # Preprocess frame
    frame_resized = cv2.resize(frame, (640, 640))  # Adjust to match model input size
    input_tensor = np.expand_dims(frame_resized, axis=0) / 255.0  # Normalize

    # Run inference
    output = infer(tf.convert_to_tensor(input_tensor, dtype=tf.float32))

    # Process model output (Assuming model outputs crash probability)
    crash_probability = output['output_0'].numpy()[0]  # Adjust key based on model output
    predicted_label = 1 if crash_probability > 0.5 else 0  # Threshold at 50%

    # Simulated ground truth labels (Replace with actual labels if available)
    true_label = 1 if "crash" in video_path.lower() else 0  # Assume crash if filename contains "crash"
    
    y_pred.append(predicted_label)
    y_true.append(true_label)

    # Display frame with prediction
    label_text = "Crash" if predicted_label == 1 else "No Crash"
    color = (0, 0, 255) if predicted_label == 1 else (0, 255, 0)
    cv2.putText(frame, f"Prediction: {label_text}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
    cv2.imshow("Frame", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Compute Performance Metrics
precision = precision_score(y_true, y_pred, zero_division=1)
recall = recall_score(y_true, y_pred, zero_division=1)
f1 = f1_score(y_true, y_pred, zero_division=1)

# Display results
print(f"\n--- Model Performance on Video ---")
print(f"Total Frames Processed: {frame_count}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")



--- Model Performance on Video ---
Total Frames Processed: 0
Precision: 1.00
Recall: 1.00
F1 Score: 1.00


In [13]:
!pip  install scikit-learn



In [15]:
!pip install tensorflow opencv-python numpy scikit-learn matplotlib seaborn 



In [16]:
import tensorflow as tf
import cv2
import numpy as np
import os
import time
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# 🔹 Load the saved TensorFlow model
model_path = "C:/vehicle-crash-detector/inference_graph/saved_model"  # Change this to your model path
if not os.path.exists(model_path):
    raise FileNotFoundError(f"Model path '{model_path}' does not exist. Check the path!")

model = tf.saved_model.load(model_path)
infer = model.signatures["serving_default"]  # Extract inference function
print("\n✅ Model loaded successfully!")

# 🔹 Print model output keys to verify correct key
print("\n🔍 Available model output keys:", infer.structured_outputs.keys())

# 🔹 Define the video path
video_path = "C:\vehicle-crash-detector\test_videos\test (1).mp4"  # Change this to your video path
if not os.path.exists(video_path):
    raise FileNotFoundError(f"Video file '{video_path}' not found. Check the path!")

cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise ValueError("❌ Unable to open video file. Check the file format and codec!")

# 🔹 Define lists to store ground truth and predicted values
y_true = []  # Actual crash labels (1 = crash, 0 = no_crash)
y_pred = []  # Model predictions

frame_count = 0
start_time = time.time()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # Stop when the video ends

    frame_count += 1

    # 🔹 Preprocess frame
    frame_resized = cv2.resize(frame, (640, 640))  # Adjust to match model input size
    input_tensor = np.expand_dims(frame_resized, axis=0) / 255.0  # Normalize

    # 🔹 Run inference
    output = infer(tf.convert_to_tensor(input_tensor, dtype=tf.float32))

    # 🔹 Process model output (Modify based on model output structure)
    output_key = list(output.keys())[0]  # Auto-detect the first output key
    crash_probability = output[output_key].numpy()[0]  # Modify if needed
    predicted_label = 1 if crash_probability > 0.5 else 0  # Threshold at 50%

    # 🔹 Simulated ground truth labels (Replace with actual labels if available)
    true_label = 1 if "crash" in video_path.lower() else 0  # Assume crash if filename contains "crash"
    
    y_pred.append(predicted_label)
    y_true.append(true_label)

    # 🔹 Display frame with prediction
    label_text = "Crash" if predicted_label == 1 else "No Crash"
    color = (0, 0, 255) if predicted_label == 1 else (0, 255, 0)
    cv2.putText(frame, f"Prediction: {label_text}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
    cv2.imshow("Vehicle Crash Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
end_time = time.time()

# 🔹 Compute Performance Metrics (Check if we have valid predictions)
if len(y_true) > 0 and len(y_pred) > 0:
    precision = precision_score(y_true, y_pred, zero_division=1)
    recall = recall_score(y_true, y_pred, zero_division=1)
    f1 = f1_score(y_true, y_pred, zero_division=1)
else:
    precision = recall = f1 = 0.0
    print("\n⚠ No valid predictions were made. Metrics may not be accurate.")

# 🔹 Compute Inference Time
total_time = end_time - start_time
avg_inference_time = total_time / frame_count if frame_count else 0

# 🔹 Print Performance Metrics
print(f"\n--- Model Performance on Video ---")
print(f"Total Frames Processed: {frame_count}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print(f"Average Inference Time per Frame: {avg_inference_time:.4f} seconds")

# 🔹 Generate Confusion Matrix (Only if data exists)
if len(y_true) > 0 and len(y_pred) > 0:
    conf_matrix = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(5, 4))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues",
                xticklabels=["No Crash", "Crash"],
                yticklabels=["No Crash", "Crash"])
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title("Confusion Matrix")
    plt.show()
else:
    print("\n⚠ No valid predictions. Skipping confusion matrix.")


ModuleNotFoundError: No module named 'sklearn'

In [ ]:
# Re-import required libraries after execution state reset
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import precision_recall_curve, average_precision_score, confusion_matrix

# Simulated ground truth and predictions (Replace with actual values)
y_true = np.random.randint(0, 2, 100)  # Replace with actual ground truth labels
y_scores = np.random.rand(100)  # Replace with model's prediction confidence scores

# Compute confusion matrix
conf_matrix = confusion_matrix(y_true, y_scores > 0.5)

# Compute precision-recall curve
precision, recall, _ = precision_recall_curve(y_true, y_scores)
average_precision = average_precision_score(y_true, y_scores)

# Plot Confusion Matrix
plt.figure(figsize=(6, 5))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap="Blues",
            xticklabels=["No Crash", "Crash"], yticklabels=["No Crash", "Crash"])
plt.xlabel("Predicted Labels")
plt.ylabel("Actual Labels")
plt.title("Confusion Matrix")
plt.show()

# Plot Precision-Recall Curve
plt.figure(figsize=(7, 5))
plt.plot(recall, precision, marker='o', linestyle='-', color='blue', label=f"AP={average_precision:.2f}")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve")
plt.legend()
plt.grid()
plt.show()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 🔹 Generate Synthetic Dataset (Simulating Reduced Visibility Conditions)
np.random.seed(42)
num_samples = 1000

# Features: Visibility (m), Vehicle Speed (m/s), Headway Distance (m)
visibility = np.random.uniform(50, 500, num_samples)  # Simulating low to high visibility
speed = np.random.uniform(10, 40, num_samples)  # Vehicle speed in m/s
distance = np.random.uniform(5, 50, num_samples)  # Headway distance in meters

# 🔹 Define Collision Risk (1 = High Risk, 0 = Low Risk)
collision_risk = ((speed / distance) > 0.8) & (visibility < 200)  # Higher risk if close and fast
collision_risk = collision_risk.astype(int)  # Convert to binary classification

# 🔹 Create DataFrame
df = pd.DataFrame({"Visibility": visibility, "Speed": speed, "Distance": distance, "CollisionRisk": collision_risk})

# 🔹 Split Data (80% Training, 20% Testing)
X = df[["Visibility", "Speed", "Distance"]]
y = df["CollisionRisk"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 🔹 Normalize Data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 🔹 Train a Neural Network (MLP Classifier)
model = MLPClassifier(hidden_layer_sizes=(10, 5), activation="relu", solver="adam", max_iter=500, random_state=42)
model.fit(X_train_scaled, y_train)

# 🔹 Predict on Test Data
y_pred = model.predict(X_test_scaled)

# 🔹 Evaluate Model Performance
accuracy = accuracy_score(y_test, y_pred)
print(f"✅ Model Accuracy: {accuracy:.2f}")
print("\n🔹 Classification Report:\n", classification_report(y_test, y_pred))

# 🔹 Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues",
            xticklabels=["No Collision", "Collision"], yticklabels=["No Collision", "Collision"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

# 🔹 Plot Decision Boundary (Visibility vs Speed)
plt.figure(figsize=(8, 6))
plt.scatter(X_test["Visibility"], X_test["Speed"], c=y_pred, cmap="coolwarm", alpha=0.7, edgecolors="k")
plt.colorbar(label="Predicted Collision Risk (0=No, 1=Yes)")
plt.xlabel("Visibility (m)")
plt.ylabel("Speed (m/s)")
plt.title("Collision Prediction Based on Visibility and Speed")
plt.show()
